import pandas as pd

In [1]:
import pandas as pd

In [2]:
averof = pd.read_csv(r'C:\Users\Dell\Desktop\Hotel\Averof Hotel.csv', error_bad_lines=False);
crown = pd.read_csv(r'C:\Users\Dell\Desktop\Hotel\Crown Inn Hotel.csv', error_bad_lines=False);

tripadv_review = pd.concat([averof, crown])
tripadv_review

,r_points,r_text,place
0,50,What a sweet hotel out the way and quite .. ca...,Averof Hotel
1,40,Stayed two nights at this hotel.Very much like...,Averof Hotel
2,40,I was on a business trip and needed a place ne...,Averof Hotel
3,10,"We booked a twin room for two nights, but when...",Averof Hotel
4,50,HiI visited Nicosia in Oct for two days and ha...,Averof Hotel
...,...,...,...
68,10,positive:area was nice and quiet.price compare...,Crown Inn Hotel
69,30,"An average quality hotel, at a cosy and quiet ...",Crown Inn Hotel
70,40,Nice hotel in a quiet area yet not so far from...,Crown Inn Hotel
71,30,I was made aware of this hotel from a list pro...,Crown Inn Hotel


In [4]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

In [5]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(token)
    return result

In [7]:
processed_reviews = tripadv_review['r_text'].map(preprocess)
processed_reviews

0     [sweet, hotel, came, visit, wanted, chill, tim...
1     [stayed, nights, hotel, liked, balcony, room, ...
2     [business, trip, needed, place, near, office, ...
3     [booked, twin, room, nights, said, room, bigge...
4     [visited, nicosia, days, interesting, city, ho...
                            ...                        
68    [positive, area, nice, quiet, price, compared,...
69    [average, quality, hotel, cosy, quiet, area, n...
70    [nice, hotel, quiet, area, center, nicosia, no...
71    [aware, hotel, list, provided, conference, org...
72    [arrived, late, night, nicosia, needed, place,...
Name: r_text, Length: 114, dtype: object

In [8]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(processed_reviews)

# Create Corpus
texts = processed_reviews

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 3), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1)]]


In [9]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=50,
                                           update_every=1,
                                           chunksize=120,
                                           passes=2,
                                           alpha='auto',
                                           eta='auto',
                                           per_word_topics=True,
                                           iterations = 4,
                                           eval_every = None)

In [10]:
from pprint import pprint

In [11]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.027*"hotel" + 0.013*"breakfast" + 0.012*"room" + 0.012*"rooms" + '
  '0.011*"good" + 0.010*"staff" + 0.010*"clean" + 0.008*"city" + 0.006*"crown" '
  '+ 0.006*"stay"'),
 (1,
  '0.030*"hotel" + 0.015*"room" + 0.013*"breakfast" + 0.012*"staff" + '
  '0.010*"good" + 0.009*"clean" + 0.009*"rooms" + 0.009*"nice" + '
  '0.008*"friendly" + 0.008*"nicosia"'),
 (2,
  '0.032*"hotel" + 0.015*"room" + 0.013*"staff" + 0.012*"breakfast" + '
  '0.012*"good" + 0.011*"friendly" + 0.011*"clean" + 0.011*"stay" + '
  '0.009*"nicosia" + 0.008*"nice"'),
 (3,
  '0.023*"hotel" + 0.018*"room" + 0.012*"clean" + 0.012*"staff" + '
  '0.012*"breakfast" + 0.011*"stay" + 0.011*"nice" + 0.010*"nicosia" + '
  '0.010*"good" + 0.008*"helpful"'),
 (4,
  '0.029*"hotel" + 0.013*"room" + 0.012*"rooms" + 0.011*"good" + '
  '0.011*"breakfast" + 0.009*"clean" + 0.009*"stay" + 0.009*"staff" + '
  '0.008*"small" + 0.008*"nicosia"')]


In [12]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.008247  0.007779       1        1  22.801165
0      0.011013  0.002831       2        1  20.049793
3     -0.005580 -0.007034       3        1  19.755230
1      0.000726 -0.000881       4        1  19.513330
4      0.002088 -0.002696       5        1  17.880480, topic_info=       Term       Freq      Total Category  logprob  loglift
216   rooms  44.000000  44.000000  Default  30.0000  30.0000
234  hotels  14.000000  14.000000  Default  29.0000  29.0000
56    small  26.000000  26.000000  Default  28.0000  28.0000
118    nice  37.000000  37.000000  Default  27.0000  27.0000
8      stay  42.000000  42.000000  Default  26.0000  26.0000
..      ...        ...        ...      ...      ...      ...
5     great   3.854031  23.517017   Topic5  -5.4050  -0.0871
118    nice   4.912053  37.774090   Topic5  -5.1624  -0.3185
126   price   3.148608  18.113161   Topic5  -5.6071  -0.0282
213    town   2.896049  15.804092   Topic5  -5.6907   0.0245
90     city   3.580402  30.535675   Topic5  -5.4786  -0.4220

[382 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
1121      5  0.447604       able
595       1  0.330743     advice
595       2  0.330743     advice
595       5  0.330743     advice
699       4  0.666498  alongside
...     ...       ...        ...
44        4  0.177106       work
44        5  0.177106       work
419       2  0.268547    working
419       3  0.268547    working
419       4  0.537094    working

[611 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 4, 2, 5])